In [12]:
import streamlit as st


import pandas as pd
import base64
import numpy as np 
import pickle
import streamlit as st
from PIL import Image
import matplotlib.pyplot as plt

#-------Deep Learning-------
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
import json 

with open(f'id2nm.json', 'r') as f:
    id2name = json.load(f)
    
with open(f'nm2id.json', 'r') as f:
    name2id = json.load(f)
    
s2m = lambda s: Chem.MolFromSmiles(s)
m2s = lambda m: Chem.MolToSmiles(m)
cancolise = lambda s: m2s(s2m(s)) 
get_dict_query = lambda q : data[str(int(q))]
get_maccs = lambda smiles: np.array(MACCSkeys.GenMACCSKeys(s2m(smiles)))
get_mrgn2 = lambda smiles: np.array(GetMorganFingerprintAsBitVect(s2m(smiles), 2, nBits=2048))
get_mrgn3 = lambda smiles: np.array(GetMorganFingerprintAsBitVect(s2m(smiles), 3, nBits=2048))



CUDA_VISIBLE_DEVICES=""



def build_isactive_model(dataset, target,balanced):
    
    print('is active model building...')
    
    #prediction using maccs fingerprint
    if balanced=="unbalance":
        # Apply RF to make predictions
        load_model_1 = pickle.load(open(f'models/{balanced}/{target}/{target}_RF_maccs_is_active.pkl', 'rb'))
    else:
        load_model_1 = pickle.load(open(f'models/{balanced}/{target}/{target}_RF_opt_balanced_maccs_is_active.pkl', 'rb'))

    prediction_1 = load_model_1.predict(dataset.mccs_fp.tolist())
    _RF_1 = f"{target}_RF_1"
    predicted_active_1 = pd.DataFrame(prediction_1, columns=[_RF_1])
    
    if balanced=="unbalance":
        # Apply ANN to make predictions
        load_model_3 = pickle.load(open(f'models/{balanced}/{target}/{target}_ANN_maccs_is_active.pkl', 'rb'))
    else:
        load_model_3 = pickle.load(open(f'models/{balanced}/{target}/{target}_ANN_opt_balanced_maccs_is_active.pkl', 'rb'))

    
    prediction_3 = load_model_3.predict(dataset.mccs_fp.tolist())
    
    _ANN_1 = f"{target}_ANN_1"
    predicted_active_3 = pd.DataFrame(prediction_3, columns=[_ANN_1])
    predicted_active_df_3 = predicted_active_1.join(predicted_active_3)
    
    #Apply model using Morgan fingerprint
    if balanced=="unbalance":
        # Apply RF to make predictions
        load_model_4 = pickle.load(open(f'models/{balanced}/{target}/{target}_RF_morgan2_is_active.pkl', 'rb'))
    else:
        load_model_4 = pickle.load(open(f'models/{balanced}/{target}/{target}_RF_opt_balanced_morgan2_is_active.pkl', 'rb'))
    prediction_4 = load_model_4.predict(dataset.mrgn_fp.tolist())
    _RF_2 = f"{target}_RF_2"
    predicted_active_4 = pd.DataFrame(prediction_4, columns=[_RF_2])
    predicted_active_df_4 = predicted_active_df_3.join(predicted_active_4)

    if balanced=="unbalance":
        # Apply ANN to make predictions
        load_model_5 = pickle.load(open(f'models/{balanced}/{target}/{target}_ANN_morgan2_is_active.pkl', 'rb'))
    else:
        load_model_5 = pickle.load(open(f'models/{balanced}/{target}/{target}_ANN_opt_balanced_morgan2_is_active.pkl', 'rb'))

    prediction_5 = load_model_5.predict(dataset.mrgn_fp.tolist())
    _ANN_2=  f"{target}_ANN_2"                                                       
    predicted_active_5 = pd.DataFrame(prediction_5, columns=[_ANN_2])
    predicted_active_df_5 = predicted_active_df_4.join(predicted_active_5)
    
    return predicted_active_df_5



# Molecular descriptor calculator
def desc_calc(input_data):
    # Performs the descriptor calculation
   input_data["mccs_fp"] = input_data["canonical_smiles"].apply(get_maccs)
   input_data["mrgn_fp"]= input_data["canonical_smiles"].apply(get_mrgn2)
   return input_data


def Predict(uploaded_file,target,balanced="unbalance"):
    # load_data.to_csv('molecule.smi', sep = '\t', header = False, index = False)
    # load_data = uploaded_file
    
    # load_data["canonical_smiles"] = load_data["SMILES"].apply(canocalis)
    # prepared_data = load_data

    dataset = desc_calc(uploaded_file)
    # Apply trained model to make prediction on query compounds
    all_df = build_isactive_model(dataset,target,balanced)
    return all_df



#Cancolise
def canocalis(smi):
    return Chem.MolToSmiles(Chem.MolFromSmiles(smi))



# # Sidebar - Sector selection
# sorted_sector_unique = sorted( df['GICS Sector'].unique() )
# selected_sector = st.sidebar.multiselect('Sector', sorted_sector_unique, sorted_sector_unique)

# # Filtering data
# df_selected_sector = df[ (df['GICS Sector'].isin(selected_sector)) ]

# st.header('Display Companies in Selected Sector')
# st.write('Data Dimension: ' + str(df_selected_sector.shape[0]) + ' rows and ' + str(df_selected_sector.shape[1]) + ' columns.')
# st.dataframe(df_selected_sector)




# File download
def filedownload(df):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()  # strings <-> bytes conversions
    href = f'<a href="data:file/csv;base64,{b64}" download="prediction.csv">Download Predictions</a>'
    return href


# st.title("Virtual sceening page")


# # Page title
# st.markdown("""
# # Predict kinase inhibitor activity \n
# ---
# """)

# sorted_sector_unique = sorted(list(id2name.keys()))
# selected_sector = st.sidebar.multiselect('Sector', sorted_sector_unique, sorted_sector_unique)

# # Sidebar
# with st.sidebar.header('1. Upload your CSV data'):
#     uploaded_file = st.sidebar.file_uploader("Upload your input file", type=['txt'])
#     st.sidebar.markdown("""
# [Example input file (same order)](./example_acetylcholinesterase.txt)
# """)

    
# if st.sidebar.button('Predict'):
#     try:
# #             file = pd.read_table(uploaded_file, sep=' ', header=0, names= ["SMILES","Name"])
#         try:
#             file = pd.read_table(uploaded_file, sep=' ', header=0, names= ["SMILES","Name"])
#             load_data = file
#             load_data["canonical_smiles"] = load_data["SMILES"].apply(canocalis)
#             loaded_data_only = load_data[:10]
#         except:
#             print("Invalid Input..!")
#         prepared_data = loaded_data_only
#         df_list =[]
        
#         for target in selected_sector:
#             df = Predict(prepared_data,target)
#             loaded_data_only = loaded_data_only.join(df)

#         combo_df = loaded_data_only.drop(['mccs_fp','mrgn_fp','canonical_smiles'], axis=1)
#         st.write(combo_df)
#         st.markdown(filedownload(combo_df), unsafe_allow_html=True)

#     except:
#         st.write("Something went wrong...!")
# else:
#     st.info('Upload input data in the sidebar to start!')
 


In [86]:
id2name

{'AAC63054': 'THRB_1',
 'AAF25870': 'PSIP1',
 'AAG43026': 'HIF1A_1',
 'AAH14840': 'SMAD2',
 'AAI00969': 'CNR1_1',
 'AAI28575': 'ESR1_1',
 'AAI36361': 'KCNJ1',
 'AAP35567': 'AL1A1',
 'AAP88778': 'HIF1A_2',
 'AAW83741': 'INS',
 'ABB72139': 'PRGR_1',
 'ADZ17337': 'RARA_1',
 'ADZ17354': 'RXRA_1',
 'AEP43755': 'ESR1_2',
 'CAG30497': 'XBP1',
 'CAG38796': 'RAD51',
 'CAI12638': 'CASP7_1',
 'EAW89041': 'IMA1',
 'EAX02111': 'FURIN',
 'EAX06628': 'JUN',
 'NP_000048': 'BLM',
 'NP_000145': 'GALK1',
 'NP_000153': 'HXK4',
 'NP_000448': 'HNF4A',
 'NP_000537': 'P53_1',
 'NP_000567': 'IL1B',
 'NP_000585': 'TNFA',
 'NP_000610': 'IFNG',
 'NP_000676': 'AGTR1',
 'NP_000698': 'V1BR',
 'NP_000729': 'ACM1',
 'NP_000919': 'PA21B',
 'NP_001005738': 'FPR2',
 'NP_001019265': 'GCR_1',
 'NP_001029027': 'DCR1C',
 'NP_001035111': 'SHPS1',
 'NP_001138783': 'PLIN1',
 'NP_001220': 'CASP9',
 'NP_001308829': 'HDAC9_1',
 'NP_001521': 'HIF1A_3',
 'NP_001531': 'HSPB1',
 'NP_001895': 'CTNB1',
 'NP_002037': 'G3P',
 'NP_002084':

In [126]:
# combo_df = combo_df[(combo_df.T != 0).any()]

/tmp/ipykernel_5508/1941257714.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data["mccs_fp"] = input_data["canonical_smiles"].apply(get_maccs)
/tmp/ipykernel_5508/1941257714.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data["mrgn_fp"]= input_data["canonical_smiles"].apply(get_mrgn2)
/tmp/ipykernel_5508/1941257714.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

is active model building...
is active model building...
is active model building...
is active model building...
is active model building...
is active model building...
is active model building...
is active model building...
is active model building...


/tmp/ipykernel_5508/1941257714.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data["mrgn_fp"]= input_data["canonical_smiles"].apply(get_mrgn2)
/tmp/ipykernel_5508/1941257714.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data["mccs_fp"] = input_data["canonical_smiles"].apply(get_maccs)
/tmp/ipykernel_5508/1941257714.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

is active model building...
is active model building...
is active model building...
is active model building...
is active model building...
is active model building...
is active model building...
is active model building...
is active model building...


/tmp/ipykernel_5508/1941257714.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data["mccs_fp"] = input_data["canonical_smiles"].apply(get_maccs)
/tmp/ipykernel_5508/1941257714.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data["mrgn_fp"]= input_data["canonical_smiles"].apply(get_mrgn2)
/tmp/ipykernel_5508/1941257714.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

...

is active model building...
is active model building...
is active model building...
is active model building...
is active model building...
is active model building...
is active model building...


/tmp/ipykernel_5508/1941257714.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data["mccs_fp"] = input_data["canonical_smiles"].apply(get_maccs)
/tmp/ipykernel_5508/1941257714.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data["mrgn_fp"]= input_data["canonical_smiles"].apply(get_mrgn2)
/tmp/ipykernel_5508/1941257714.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

is active model building...
is active model building...
is active model building...
is active model building...


/tmp/ipykernel_5508/1941257714.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data["mccs_fp"] = input_data["canonical_smiles"].apply(get_maccs)
/tmp/ipykernel_5508/1941257714.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data["mrgn_fp"]= input_data["canonical_smiles"].apply(get_mrgn2)
/tmp/ipykernel_5508/1941257714.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

is active model building...
is active model building...
is active model building...
is active model building...


In [127]:
combo_df.to_csv('result__.csv',index=None)

In [128]:
id2name['5HT1B']

KeyError: '5HT1B'

In [214]:
s1 = pd.DataFrame([{'a':1,'b':1}], index=[2])
s2 = pd.DataFrame([{'a':1,'b':0}], index=[2])   
# df_all =pd.concat([s1, s2],axis=1)
df_all
# s1 = s1.join(s2)
# s1 = s1[(s1.T != 0).any()]
pd.concat([s1,s2],axis=0)

,a,b
2,1,1
2,1,0


In [118]:

with open(f'smiles_cid_washed_final.json', 'r') as f:
    cid_valid = list(json.load(f).values())[:10]

cid_valid

['CC(O)CN',
 'O=[N+]([O-])c1ccc(Cl)c([N+](=O)[O-])c1',
 'CCn1cnc2c(N)ncnc21',
 'ClCCCl',
 'Clc1ccc(Cl)c(Cl)c1',
 'O=C(O)c1cccc(O)c1O',
 'O=CCCl',
 'OCCCl',
 'OCC1OC(O)CC(O)C1O',
 'O=C(O)CCC(=O)C(=O)O']

In [121]:
ls

'1_virtual_screen (copy).py'   nm2id.json
 1_virtual_screen.py           O00264.csv
 App.ipynb                     O00264.csv.gz
 app.py                        Other/
'example (copy).txt'           P51817.csv
 example.txt                   P51817.csv.gz
 id2name_byhand.json           requirement.ipynb
 id2nm.json                    smiles_cid_washed_final.json
'is_active (copy).ipynb'       sp500-app.py
 ML_example.ipynb              Untitled.ipynb
 models/


In [123]:
import pandas as pd 

df = pd.read_csv('P51817.csv.gz')
set(df.cid.tolist())

{2044,
 2048,
 2049,
 2051,
 2063,
 2396,
 2399,
 2422,
 2427,
 2703,
 2856,
 2857,
 3499,
 3501,
 3547,
 3707,
 3795,
 3797,
 3820,
 3971,
 3973,
 4665,
 4707,
 4709,
 4712,
 4713,
 4879,
 5034,
 5162,
 5164,
 5169,
 5174,
 5228,
 5291,
 8515,
 44259,
 65399,
 73293,
 108047,
 123631,
 126274,
 126565,
 151194,
 153999,
 156414,
 156422,
 160355,
 176155,
 176167,
 176870,
 189883,
 208908,
 216239,
 290221,
 389898,
 438981,
 447077,
 447966,
 448008,
 448014,
 448042,
 448912,
 448949,
 449054,
 449241,
 451705,
 453216,
 456214,
 481747,
 509554,
 579429,
 599038,
 644241,
 644354,
 657356,
 672296,
 753704,
 766948,
 766949,
 1048845,
 1474853,
 1893668,
 2326992,
 2794188,
 2858522,
 3004085,
 3025986,
 3035817,
 3038522,
 3038525,
 3062316,
 3064778,
 3081361,
 3323623,
 3385203,
 3387354,
 3467590,
 3641059,
 3659633,
 3805075,
 4124851,
 4521392,
 5005498,
 5025739,
 5113385,
 5154691,
 5278396,
 5284616,
 5287855,
 5287969,
 5289419,
 5310991,
 5311510,
 5312122,
 5312126,
 5

In [135]:
int((1/20)*100)

5

In [138]:
targ_dict = {o:i for i,o in zip(range(5),[x for x in range(5)])}
targ_dict

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4}

In [206]:
combo_df[:1]

,SMILES,Name,AAC63054_RF_1,AAC63054_ANN_1,AAC63054_RF_2,AAC63054_ANN_2,AAF25870_RF_1,AAF25870_ANN_1,AAF25870_RF_2,AAF25870_ANN_2,...,Q9Y6E0_RF_2,Q9Y6E0_ANN_2,Q9Y6M4_RF_1,Q9Y6M4_ANN_1,Q9Y6M4_RF_2,Q9Y6M4_ANN_2,Q9Y6R4_RF_1,Q9Y6R4_ANN_1,Q9Y6R4_RF_2,Q9Y6R4_ANN_2
0,CC(C)(C)C1=C(C(=CC(=C1)C=C(C#N)C#N)Br)O,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [207]:
combo_df[:1]

0    1
Name: ABB72139_ANN_1, dtype: int64

In [218]:
[x for x in range(len(combo_df))]

[0, 1, 2, 3, 4, 5]

In [237]:
def rename(x):
    if len(x.split('_')) >=3:
        return x.split('_')[:2]
    else:
        return x.split('_')[:1]


In [238]:
['_'.join(rename(x)) for x in ['P20248_1_RF_2','P20248_ANN_2']]

['P20248_1', 'P20248']

In [239]:
len('P20248_1_RF_2'.split('_')) 

4

In [ ]:
P20248_1_RF_2

In [10]:
def add_target_columns(combo_df):
    rename = lambda x : x.split('_')[:2] if len(x.split('_')) >=4 else x.split('_')[:1]
    models = {}
    for index in range(len(combo_df)):
        d = combo_df.T
        d = d[index]
        models[index] = ','.join(list(set([rename(x)[0] for x in list(d[d==1].index)])))
        df = pd.DataFrame([models]).T
        df= df.rename(columns={
            0:'Target'
        })

        df['target_number'] = df.Target.apply(lambda x : len(x.split(',')))
        final_df = combo_df.join(df)
    return final_df

In [286]:
final_df = add_target_columns(combo_df)

,SMILES,Name,AAC63054_RF_1,AAC63054_ANN_1,AAC63054_RF_2,AAC63054_ANN_2,AAF25870_RF_1,AAF25870_ANN_1,AAF25870_RF_2,AAF25870_ANN_2,...,Q9Y6M4_RF_1,Q9Y6M4_ANN_1,Q9Y6M4_RF_2,Q9Y6M4_ANN_2,Q9Y6R4_RF_1,Q9Y6R4_ANN_1,Q9Y6R4_RF_2,Q9Y6R4_ANN_2,Target,target_number
0,CC(C)(C)C1=C(C(=CC(=C1)C=C(C#N)C#N)Br)O,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"P05771,P04150,P41743,Q8IU85,P12931,P54646,Q8IX...",214
1,CN(C)CCCN1C=C(C2=C1C=CC(=C2)OC)C3=C(C(=O)NC3=O...,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"P05771,Q02779,P04150,P41743,Q8IU85,P12931,P546...",227
2,C1COCCN1C2=CC(=O)C3=C(O2)C(=CC=C3)C4=CC=CC=C4,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"P04150,P12931,Q8IXJ6,Q9UM73,P35968,P30530,AEP4...",168
3,CS(=O)(=O)CCNCC1=CC=C(O1)C2=CC3=C(C=C2)N=CN=C3...,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"P04150,P12931,P54646,Q8IXJ6,P07947,Q9UM73,P359...",162
4,C1=CC(=CC=C1C2=CSC(=N2)NC3=CC=C(C=C3)O)Cl,6,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,"P04150,Q9HCP0,P12931,P54646,Q8IXJ6,P07947,Q9UM...",192
5,N#CCCNC[C@H]1CC[C@H](c2nnn3cnc4[nH]ccc4c23)CC1,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"P05771,P46734,P41743,Q8IU85,P12931,P54646,Q8IX...",217


,SMILES,Name,AAC63054_RF_1,AAC63054_ANN_1,AAC63054_RF_2,AAC63054_ANN_2,AAF25870_RF_1,AAF25870_ANN_1,AAF25870_RF_2,AAF25870_ANN_2,...,Q9Y6M4_RF_1,Q9Y6M4_ANN_1,Q9Y6M4_RF_2,Q9Y6M4_ANN_2,Q9Y6R4_RF_1,Q9Y6R4_ANN_1,Q9Y6R4_RF_2,Q9Y6R4_ANN_2,Target,target_number
0,CC(C)(C)C1=C(C(=CC(=C1)C=C(C#N)C#N)Br)O,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"P05771,P04150,P41743,Q8IU85,P12931,P54646,Q8IX...",214
1,CN(C)CCCN1C=C(C2=C1C=CC(=C2)OC)C3=C(C(=O)NC3=O...,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"P05771,Q02779,P04150,P41743,Q8IU85,P12931,P546...",227
2,C1COCCN1C2=CC(=O)C3=C(O2)C(=CC=C3)C4=CC=CC=C4,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"P04150,P12931,Q8IXJ6,Q9UM73,P35968,P30530,AEP4...",168
3,CS(=O)(=O)CCNCC1=CC=C(O1)C2=CC3=C(C=C2)N=CN=C3...,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"P04150,P12931,P54646,Q8IXJ6,P07947,Q9UM73,P359...",162
4,C1=CC(=CC=C1C2=CSC(=N2)NC3=CC=C(C=C3)O)Cl,6,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,"P04150,Q9HCP0,P12931,P54646,Q8IXJ6,P07947,Q9UM...",192
5,N#CCCNC[C@H]1CC[C@H](c2nnn3cnc4[nH]ccc4c23)CC1,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"P05771,P46734,P41743,Q8IU85,P12931,P54646,Q8IX...",217


In [251]:
s1 = pd.DataFrame([{'a':1,'b':1}], index=[2])
s2 = pd.DataFrame([{'a':1,'b':0}], index=[2])   
# df_all =pd.concat([s1, s2],axis=1)
df_all
# s1 = s1.join(s2)
# s1 = s1[(s1.T != 0).any()]
pd.concat([s1,s2],axis=1)

,a,b,a,b
2,1,1,1,0


In [8]:
file = pd.read_table('example.txt', sep=' ', header=0, names= ["SMILES","Name"])
load_data = file
load_data["canonical_smiles"] = load_data["SMILES"].apply(canocalis)
loaded_data_only = load_data[:10]

prepared_data = loaded_data_only
df_list =[]

for target in sorted(list(id2name.keys())):
    df = Predict(prepared_data,target,'balance_opt')
    loaded_data_only = loaded_data_only.join(df)
combo_df = loaded_data_only.drop(['mccs_fp','mrgn_fp','canonical_smiles'], axis=1)


/tmp/ipykernel_44106/4243361323.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data["mccs_fp"] = input_data["canonical_smiles"].apply(get_maccs)
/tmp/ipykernel_44106/4243361323.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data["mrgn_fp"]= input_data["canonical_smiles"].apply(get_mrgn2)


is active model building...


/tmp/ipykernel_44106/4243361323.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data["mccs_fp"] = input_data["canonical_smiles"].apply(get_maccs)
/tmp/ipykernel_44106/4243361323.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data["mrgn_fp"]= input_data["canonical_smiles"].apply(get_mrgn2)
/tmp/ipykernel_44106/4243361323.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

is active model building...
is active model building...
is active model building...
is active model building...
is active model building...
is active model building...
is active model building...
is active model building...


/tmp/ipykernel_44106/4243361323.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data["mccs_fp"] = input_data["canonical_smiles"].apply(get_maccs)
/tmp/ipykernel_44106/4243361323.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data["mrgn_fp"]= input_data["canonical_smiles"].apply(get_mrgn2)
/tmp/ipykernel_44106/4243361323.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

...

/tmp/ipykernel_44106/4243361323.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data["mccs_fp"] = input_data["canonical_smiles"].apply(get_maccs)
/tmp/ipykernel_44106/4243361323.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data["mrgn_fp"]= input_data["canonical_smiles"].apply(get_mrgn2)
/tmp/ipykernel_44106/4243361323.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

is active model building...
is active model building...
is active model building...
is active model building...
is active model building...
is active model building...


/tmp/ipykernel_44106/4243361323.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data["mccs_fp"] = input_data["canonical_smiles"].apply(get_maccs)
/tmp/ipykernel_44106/4243361323.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data["mrgn_fp"]= input_data["canonical_smiles"].apply(get_mrgn2)
/tmp/ipykernel_44106/4243361323.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

is active model building...
is active model building...
is active model building...
is active model building...
is active model building...
is active model building...
is active model building...


/tmp/ipykernel_44106/4243361323.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data["mccs_fp"] = input_data["canonical_smiles"].apply(get_maccs)
/tmp/ipykernel_44106/4243361323.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data["mrgn_fp"]= input_data["canonical_smiles"].apply(get_mrgn2)


In [11]:
final_df = add_target_columns(combo_df)
final_df

,SMILES,Name,AAC63054_RF_1,AAC63054_ANN_1,AAC63054_RF_2,AAC63054_ANN_2,AAF25870_RF_1,AAF25870_ANN_1,AAF25870_RF_2,AAF25870_ANN_2,...,Q9Y6M4_RF_1,Q9Y6M4_ANN_1,Q9Y6M4_RF_2,Q9Y6M4_ANN_2,Q9Y6R4_RF_1,Q9Y6R4_ANN_1,Q9Y6R4_RF_2,Q9Y6R4_ANN_2,Target,target_number
0,COc1ccc(F)c(Cl)c1C(C)Oc1cc(-c2cnn(C3CCNCC3)c2)...,CHEMBL4461078,1,0,0,0,1,1,0,1,...,0,1,1,0,0,0,0,1,"P50750,P51813,EAW89041,Q16659,Q9BXA7,Q15835,P4...",313
1,O=C(NCC(F)(F)F)[C@H]1CC[C@H](c2nnn3cnc4[nH]ccc...,CHEMBL3894516,1,1,0,1,0,0,0,0,...,1,0,0,0,0,0,0,1,"P10398,O00311,Q13177,P50750,Q9BRS2,Q9UBN7,P570...",259
2,N#C[C@@H]1CCN(Cc2cnccn2)C[C@H]1n1cc(C(N)=O)c(N...,CHEMBL3944948,1,0,0,1,0,1,0,0,...,0,0,0,0,1,1,0,1,"P31751,O00311,Q13177,P50750,Q9BRS2,Q9UBN7,P570...",281
3,CC(Nc1nc2cc[nH]c(=O)c2c2cc(F)ccc12)C(C)(C)C,CHEMBL1290309,0,0,1,0,0,0,0,1,...,0,1,1,0,0,1,0,1,"P10398,O00311,Q13177,P50750,Q9BRS2,Q96SB4,Q9UB...",292
4,N#C[C@H]1C[C@H](O)CC[C@@H]1n1cc(C(N)=O)c(Nc2cc...,CHEMBL3702940,1,1,1,0,0,1,0,0,...,0,0,1,0,0,0,0,1,"P31751,O00311,Q9UBN7,Q9BRS2,P57059,P21730,Q061...",274
5,N#CCCNC[C@H]1CC[C@H](c2nnn3cnc4[nH]ccc4c23)CC1,CHEMBL3894474,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,1,"O00311,Q13177,P50750,P51813,EAW89041,P19784,Q1...",314


In [298]:
final_df['P04150_RF_1']

0    1
1    1
2    0
3    1
4    1
5    0
Name: P04150_RF_1, dtype: int64

In [ ]:
final_df = add_target_columns(combo_df)
final_df

In [149]:
combo_df = loaded_data_only.drop(['mccs_fp','mrgn_fp','canonical_smiles'], axis=1)
combo_df

,SMILES,Name,AAC63054_RF_1,AAC63054_ANN_1,AAC63054_RF_2,AAC63054_ANN_2,AAF25870_RF_1,AAF25870_ANN_1,AAF25870_RF_2,AAF25870_ANN_2,...,Q9Y6E0_RF_2,Q9Y6E0_ANN_2,Q9Y6M4_RF_1,Q9Y6M4_ANN_1,Q9Y6M4_RF_2,Q9Y6M4_ANN_2,Q9Y6R4_RF_1,Q9Y6R4_ANN_1,Q9Y6R4_RF_2,Q9Y6R4_ANN_2
0,CC(C)(C)C1=C(C(=CC(=C1)C=C(C#N)C#N)Br)O,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CN(C)CCCN1C=C(C2=C1C=CC(=C2)OC)C3=C(C(=O)NC3=O...,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,C1COCCN1C2=CC(=O)C3=C(O2)C(=CC=C3)C4=CC=CC=C4,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,CS(=O)(=O)CCNCC1=CC=C(O1)C2=CC3=C(C=C2)N=CN=C3...,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,C1=CC(=CC=C1C2=CSC(=N2)NC3=CC=C(C=C3)O)Cl,6,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
5,N#CCCNC[C@H]1CC[C@H](c2nnn3cnc4[nH]ccc4c23)CC1,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [190]:
col = []
combo_df = loaded_data_only.drop(['mccs_fp','mrgn_fp','canonical_smiles'], axis=1)

df_copy = combo_df.copy()

for x in list(loaded_data_only.columns)[5:]:
    df = df_copy[df_copy[x] ==1]
    if not df.empty:
        col.append(x)
    else:
        continue

# df = loaded_data_only.copy()
    
l = df_copy[df_copy['AAI36361_ANN_2'] ==1]
l= l.apply(lambda row : row, axis=1)
l
# col

,SMILES,Name,AAC63054_RF_1,AAC63054_ANN_1,AAC63054_RF_2,AAC63054_ANN_2,AAF25870_RF_1,AAF25870_ANN_1,AAF25870_RF_2,AAF25870_ANN_2,...,Q9Y6E0_RF_2,Q9Y6E0_ANN_2,Q9Y6M4_RF_1,Q9Y6M4_ANN_1,Q9Y6M4_RF_2,Q9Y6M4_ANN_2,Q9Y6R4_RF_1,Q9Y6R4_ANN_1,Q9Y6R4_RF_2,Q9Y6R4_ANN_2
2,C1COCCN1C2=CC(=O)C3=C(O2)C(=CC=C3)C4=CC=CC=C4,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [289]:
# [x.split('_')[0] for x in list(loaded_data_only.columns)[5:]]

In [ ]:
st.write(combo_df)
st.markdown(filedownload(combo_df), unsafe_allow_html=True)
